# Summary

This notebook calculates price growths within a specified horizon from every week and bins them into discrete levels, to be used as the multi-label target variable for a prediction task.

# Imports and configuration

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd

from typing import List, Tuple

In [2]:
sys.path.append(r"C:\Users\mushj\OneDrive\Desktop\WORK\Financial Analytics\pipeline")

from fin_ml.data_engineering.discretize_price_growth import engineer_targets, generate_symmetric_intervals

In [3]:
# configuration
HORIZON_DAYS = 90
HORIZON_MARGIN = 5
PERSISTENCE = 5

INPUT_PATH = "C:/Users/mushj/Downloads/PROCESSED FINANCE DATA/FMP"
OUTPUT_PATH = "C:/Users/mushj/Downloads/CURATED FINANCE DATA/FMP"

In [4]:
# load historical daily prices
df = pd.read_csv(INPUT_PATH+'/FMP_daily_prices_top1k.csv')

# load list of symbols to keep (based on exploratory analysis)
with open(INPUT_PATH+'/top1k_subset', 'rb') as f:
    include_symbols = pickle.load(f)

# Prepare DataFrame

In [5]:
print("Number of symbols to include:", len(include_symbols))
print("Examples:", include_symbols[:5])

Number of symbols to include: 978
Examples: ['A', 'AA', 'AAL', 'AAON', 'AAP']


In [6]:
df = df.query("symbol in @include_symbols")
df['date'] = pd.to_datetime(df['date'])

In [7]:
df.head()

,symbol,date,close,volume
0,A,2005-01-03,16.09,3587208.0
1,A,2005-01-04,15.66,3978002.0
2,A,2005-01-05,15.66,4139634.0
3,A,2005-01-06,15.31,3353443.0
4,A,2005-01-07,15.30,2786175.0


# Engineer multi-label target variable

1. The daily stock price data is downsampled to a weekly frequency. The last daily closing price is used as the week's closing price. For example, if the week contains prices for Monday till Friday, then Friday's closing price is used.
2. Given a forecast horizon (e.g. 1 year ahead), price growths are computed for each week, between a week's closing price and EVERY subsequent DAILY closing price within the forecast horizon.
3. Discrete intervals are defined, for example, <-25%, -25to0%, 0to25% >25%. The multi-label target variable for each week is a vector of binary values that indicate whether a stock will experience a certain level of price growth within the given horizon from that week. For example, a vector of [0, 1, 0, 1] for week 50 of stock ABC means that the stock will experience at least one price change in the interval -25to0%, and at least one price change in the interval >25%, relative to the closing price of week 50.
4. A persistence parameter controls the 'strictness' of accounting for price changes. It specifies the minimum number of consecutive trading days for which a price growth must be sustained. For example, if persistence=3, then a price growth of 30% must be sustained for 3 consecutive trading days before the label can be set to 1.

## Get weekly data

In [8]:
%%time
weekly_df = (
    df
    .sort_values(['symbol', 'date'])
    .groupby('symbol')
    .resample('W-MON', on='date', label='left') # mark each week with Monday
    .last() # get last price of the week
    .reset_index(level='date') # set date index as column
    .reset_index(drop=True) # remove other indexes
    .rename({'date': 'week'}, axis=1)
)

CPU times: total: 10.3 s
Wall time: 10.5 s


In [9]:
weekly_df.head()

,week,symbol,close,volume
0,2004-12-27,A,16.09,3587208.0
1,2005-01-03,A,15.23,4190098.0
2,2005-01-10,A,14.86,3027509.0
3,2005-01-17,A,14.45,5241370.0
4,2005-01-24,A,14.90,2821500.0


In [10]:
weekly_df.isna().mean()

week      0.0
symbol    0.0
close     0.0
volume    0.0
dtype: float64

In [11]:
weekly_df.dtypes

week      datetime64[ns]
symbol            object
close            float64
volume           float64
dtype: object

In [12]:
# distribution of counts of weekly data points
weekly_df.value_counts('symbol').describe()

count     978.000000
mean      871.982618
std       282.940932
min       155.000000
25%       709.250000
50%      1045.000000
75%      1045.000000
max      1045.000000
Name: count, dtype: float64

## Validate weekly data

In [13]:
%%time
# store the complete sets of weeks between the start and end weeks of each symbol (span)
week_span = {}

for symbol, data in weekly_df.groupby('symbol'):
    weeks = data.query("symbol == @symbol")['week']
    start, end = weeks.min(), weeks.max()
    week_span[symbol] = set(pd.date_range(start, end, freq='W-MON'))

CPU times: total: 10.8 s
Wall time: 11.1 s


In [14]:
%%time
# detect missing weeks for each symbol
missing_weeks = []

for symbol, data in weekly_df.groupby('symbol'):
    # get weeks in symbol span that are not in symbol data
    missing = week_span[symbol].difference(set(data['week']))
    for w in missing:
        missing_weeks.append({'symbol': symbol, 'missing_week': w})
        
missing_weeks = pd.DataFrame(missing_weeks)

CPU times: total: 672 ms
Wall time: 692 ms


In [15]:
# check missing weeks
missing_weeks

""


## Set intervals

In [16]:
# log-spaced price changes between 1% and 15%
boundaries = np.logspace(np.log(0.02), np.log(0.15), num=4, base=np.e)
np.round(boundaries, 3)

array([0.02 , 0.039, 0.077, 0.15 ])

In [17]:
boundaries = [0, 0.02, 0.039, 0.077, 0.15, np.inf]
intervals = generate_symmetric_intervals(boundaries)

print(len(intervals))
for i in intervals:
    print(i)

10
(-inf, -0.15)
(-0.15, -0.077)
(-0.077, -0.039)
(-0.039, -0.02)
(-0.02, 0)
(0, 0.02)
(0.02, 0.039)
(0.039, 0.077)
(0.077, 0.15)
(0.15, inf)


## Compute price changes

In [18]:
weekly_df = weekly_df.rename({'week': 'date'}, axis=1)

In [19]:
directory = OUTPUT_PATH+f'/discrete_return_{HORIZON_DAYS}d_{PERSISTENCE}p'
if not os.path.exists(directory):
    os.makedirs(directory)
    print('Created', directory)
else:
    pass

In [20]:
with open(f'{directory}/labels', 'wb') as f:
    pickle.dump(intervals, f)

In [21]:
%%time
# calculate for each symbol separately
for symbol, data in weekly_df.groupby('symbol'):
    target_df = engineer_targets(
        data, df, 
        forecast_horizon=HORIZON_DAYS, 
        horizon_margin=HORIZON_MARGIN, 
        intervals=intervals,
        persistence=PERSISTENCE
    )
    target_df.to_csv(f'{directory}/{symbol}.csv', index=False)

Calculating for APLS: 100%|█████████████████████████████████████████████████████████| 374/374 [00:00<00:00, 479.23it/s]


Calculating for BSY: 100%|██████████████████████████████████████████████████████████| 224/224 [00:00<00:00, 484.40it/s]


Calculating for COIN: 100%|█████████████████████████████████████████████████████████| 195/195 [00:00<00:00, 496.40it/s]


Calculating for DOCS: 100%|█████████████████████████████████████████████████████████| 185/185 [00:00<00:00, 503.03it/s]


Calculating for FBIN: 100%|█████████████████████████████████████████████████████████| 695/695 [00:01<00:00, 366.46it/s]


...

Calculating for STAG: 100%|█████████████████████████████████████████████████████████| 717/717 [00:02<00:00, 339.16it/s]


Calculating for UBER: 100%|█████████████████████████████████████████████████████████| 296/296 [00:00<00:00, 493.48it/s]


Calculating for WST: 100%|████████████████████████████████████████████████████████| 1045/1045 [00:02<00:00, 356.40it/s]


Calculating for ZTS: 100%|██████████████████████████████████████████████████████████| 623/623 [00:01<00:00, 365.78it/s]

CPU times: total: 21min 22s
Wall time: 40min 32s


## Sample result

In [22]:
target_df

,symbol,week,labels
0,ZTS,2013-01-28,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 0]"
1,ZTS,2013-02-04,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 0]"
2,ZTS,2013-02-11,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
3,ZTS,2013-02-18,"[0, 0, 1, 0, 1, 1, 1, 1, 1, 0]"
4,ZTS,2013-02-25,"[0, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
...,...,...,...
618,ZTS,2024-12-02,NaN
619,ZTS,2024-12-09,NaN
620,ZTS,2024-12-16,NaN
621,ZTS,2024-12-23,NaN


In [23]:
t_df = pd.DataFrame(target_df.dropna()['labels'].to_list())
t_df.columns = intervals
t_df.describe()

,"(-inf, -0.15)","(-0.15, -0.077)","(-0.077, -0.039)","(-0.039, -0.02)","(-0.02, 0)","(0, 0.02)","(0.02, 0.039)","(0.039, 0.077)","(0.077, 0.15)","(0.15, inf)"
count,611.000000,611.000000,611.000000,611.000000,611.0,611.00000,611.000000,611.000000,611.000000,611.000000
mean,0.111293,0.330606,0.549918,0.772504,1.0,0.97054,0.857610,0.780687,0.541735,0.202946
std,0.314752,0.470817,0.497910,0.419559,0.0,0.16923,0.349736,0.414120,0.498663,0.402522
min,0.000000,0.000000,0.000000,0.000000,1.0,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.0,1.00000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,1.000000,1.0,1.00000,1.000000,1.000000,1.000000,0.000000
75%,0.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.000000,1.000000,1.000000,1.000000
